In [1]:
import sys
import os
import random
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle
from somhos.config.paths import get_relative_path
from somhos.config.paths import DOCS_SAMPLE_A_SUFFIX, DOCS_SAMPLE_B_SUFFIX
from somhos.config.paths import DOCS_SAMPLES_CONTENT
from somhos.config.paths import DOCS_SAMPLES_WORD_DOC_COUNT, DOCS_SAMPLES_WORD_COUNT
from somhos.config.paths import DOCS_SAMPLES_KPS_DOC_COUNT, DOCS_SAMPLES_KPS_COUNT
from somhos.config.paths import DOCS_SAMPLES_JACCARD_SIM, DOCS_SAMPLES_JACCARD_SIM_UDV
from somhos.config.paths import DOCS_SAMPLES_JACCARD_SIM_KPS, DOCS_SAMPLES_JACCARD_SIM_UDV_KPS
from somhos.config.paths import DOCS_SAMPLES_TF_IDF, KPS_DIRECTORY_INVERSE_SUFFIX

Default path

In [2]:
data_path = "../../src/somhos/resources/aminer/v9"

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (1000, 1000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 2000


In [5]:
from nltk.metrics.distance import jaccard_distance
from itertools import product
import numpy as np

In [6]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_kps(index, samples_content):
    return samples_content[index]['bag-of-kps']

m_jaccard_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    set_a = get_kps(docs_sample_a[i], samples_content)
    set_b = get_kps(docs_sample_b[j], samples_content)
    if set_a and set_b:  
        jaccard_sim = 1 - jaccard_distance(set_a, set_b)
    else:
        jaccard_sim = 0
    m_jaccard_sim[i, j] = jaccard_sim

print(m_jaccard_sim)

[[0.         0.         0.         ... 0.01960784 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.01785714 0.         0.02857143]
 ...
 [0.         0.         0.         ... 0.         0.03846154 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.02040816 0.         0.         ... 0.         0.         0.        ]]


In [7]:
docs_samples_jaccard_sim_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM_KPS)
save_pickle(m_jaccard_sim, docs_samples_jaccard_sim_path)

In [8]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_jaccard_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_jaccard_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_jaccard_sim)

[[0.00088732 0.00074947 0.00072513 ... 0.00090514 0.00039251 0.00080765]
 [0.00178591 0.00150847 0.00145948 ... 0.00182178 0.00079    0.00162555]
 [0.00490652 0.00414428 0.00400969 ... 0.00500508 0.00217041 0.00446596]
 ...
 [0.00330685 0.00279312 0.00270241 ... 0.00337327 0.00146279 0.00300993]
 [0.00058503 0.00049415 0.0004781  ... 0.00059678 0.00025879 0.0005325 ]
 [0.00360416 0.00304425 0.00294538 ... 0.00367656 0.00159431 0.00328055]]


In [9]:
docs_samples_jaccard_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM_UDV_KPS)
save_pickle(m_udv_jaccard_sim, docs_samples_jaccard_sim_udv_path)